In [1]:
# Import required libraries
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from bert_score import score
import pandas as pd


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

def load_gpt2_model():
    print("Loading GPT-2 model...")
    model_name = "gpt2"  # Using the smallest GPT-2 model for efficiency
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    return model, tokenizer

# Load the model
model, tokenizer = load_gpt2_model()

Loading GPT-2 model...


In [3]:
# ## Part 2 - Design Your Prompts and Generate Outputs

def generate_motivational_text(prompt, model, tokenizer, num_outputs=3, max_length=50):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = []
    
    for _ in range(num_outputs):
        # Generate text
        output = model.generate(
            inputs, 
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        outputs.append(generated_text)
    
    return outputs

In [4]:
# Define 5 distinct prompts
prompts = [
    {
        "type": "Direct instruction",
        "text": "Write a motivational quote about overcoming fear."
    },
    {
        "type": "Scenario-based",
        "text": "Imagine you're helping a friend who failed a test. Write something encouraging."
    },
    {
        "type": "Persona-based",
        "text": "As a wise monk, write a quote about inner strength."
    },
    {
        "type": "Keyword-based",
        "text": "Using the words 'growth', 'struggle', and 'hope', write something inspiring."
    },
    {
        "type": "Conversational",
        "text": "User: I feel like giving up. GPT-2: Here's a quote for you:"
    }
]

# Generate outputs for each prompt
all_outputs = []
for prompt in prompts:
    outputs = generate_motivational_text(prompt["text"], model, tokenizer)
    for i, output in enumerate(outputs):
        all_outputs.append({
            "Prompt Type": prompt["type"],
            "Prompt Text": prompt["text"],
            "Output #": i+1,
            "Generated Text": output
        })

outputs_df = pd.DataFrame(all_outputs)
outputs_df


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask

,Prompt Type,Prompt Text,Output #,Generated Text
0,Direct instruction,Write a motivational quote about overcoming fear.,1,Write a motivational quote about overcoming fe...
1,Direct instruction,Write a motivational quote about overcoming fear.,2,Write a motivational quote about overcoming fe...
2,Direct instruction,Write a motivational quote about overcoming fear.,3,Write a motivational quote about overcoming fe...
3,Scenario-based,Imagine you're helping a friend who failed a t...,1,Imagine you're helping a friend who failed a t...
4,Scenario-based,Imagine you're helping a friend who failed a t...,2,Imagine you're helping a friend who failed a t...
5,Scenario-based,Imagine you're helping a friend who failed a t...,3,Imagine you're helping a friend who failed a t...
6,Persona-based,"As a wise monk, write a quote about inner stre...",1,"As a wise monk, write a quote about inner stre..."
7,Persona-based,"As a wise monk, write a quote about inner stre...",2,"As a wise monk, write a quote about inner stre..."
8,Persona-based,"As a wise monk, write a quote about inner stre...",3,"As a wise monk, write a quote about inner stre..."
9,Keyword-based,"Using the words 'growth', 'struggle', and 'hop...",1,"Using the words 'growth', 'struggle', and 'hop..."


In [5]:
## Part 3 - Get a Human-Written Reference

# %%
def get_human_reference():
    # This would normally be collected from an external source
    human_reference = {
        "text": "The only way to overcome fear is to face it. Every struggle you endure plants the seeds for future growth. Have hope, for even the darkest night will end and the sun will rise.",
        "source": "https://example.com/motivational-quotes"  # Example URL
    }
    return human_reference

human_reference = get_human_reference()

print("Human Reference Text:")
print(human_reference["text"])
print(f"\nSource: {human_reference['source']}")

Human Reference Text:
The only way to overcome fear is to face it. Every struggle you endure plants the seeds for future growth. Have hope, for even the darkest night will end and the sun will rise.

Source: https://example.com/motivational-quotes


In [6]:
## Part 4 - Evaluate Outputs Using BERTScore

# %%
def evaluate_with_bertscore(generated_texts, human_reference):
    # Compute BERTScore
    P, R, F1 = score(generated_texts, [human_reference["text"]] * len(generated_texts), lang="en", verbose=True)
    
    # Convert to list for easier handling
    return F1.tolist()

# %%
generated_texts = outputs_df["Generated Text"].tolist()
f1_scores = evaluate_with_bertscore(generated_texts, human_reference)

# Add BERTScore to dataframe
outputs_df["BERTScore F1"] = f1_scores
outputs_df


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 200.04it/s]

done in 1.22 seconds, 12.34 sentences/sec


,Prompt Type,Prompt Text,Output #,Generated Text,BERTScore F1
0,Direct instruction,Write a motivational quote about overcoming fear.,1,Write a motivational quote about overcoming fe...,0.853215
1,Direct instruction,Write a motivational quote about overcoming fear.,2,Write a motivational quote about overcoming fe...,0.842215
2,Direct instruction,Write a motivational quote about overcoming fear.,3,Write a motivational quote about overcoming fe...,0.844847
3,Scenario-based,Imagine you're helping a friend who failed a t...,1,Imagine you're helping a friend who failed a t...,0.840076
4,Scenario-based,Imagine you're helping a friend who failed a t...,2,Imagine you're helping a friend who failed a t...,0.832974
5,Scenario-based,Imagine you're helping a friend who failed a t...,3,Imagine you're helping a friend who failed a t...,0.836215
6,Persona-based,"As a wise monk, write a quote about inner stre...",1,"As a wise monk, write a quote about inner stre...",0.857517
7,Persona-based,"As a wise monk, write a quote about inner stre...",2,"As a wise monk, write a quote about inner stre...",0.841372
8,Persona-based,"As a wise monk, write a quote about inner stre...",3,"As a wise monk, write a quote about inner stre...",0.859363
9,Keyword-based,"Using the words 'growth', 'struggle', and 'hop...",1,"Using the words 'growth', 'struggle', and 'hop...",0.837028


In [7]:
## Part 5 - Results Table and Submission Checklist

# %%
# Create final results table
results_table = outputs_df[["Prompt Type", "Output #", "BERTScore F1", "Generated Text"]]
print("=== Final Results ===")
display(results_table)

=== Final Results ===


,Prompt Type,Output #,BERTScore F1,Generated Text
0,Direct instruction,1,0.853215,Write a motivational quote about overcoming fe...
1,Direct instruction,2,0.842215,Write a motivational quote about overcoming fe...
2,Direct instruction,3,0.844847,Write a motivational quote about overcoming fe...
3,Scenario-based,1,0.840076,Imagine you're helping a friend who failed a t...
4,Scenario-based,2,0.832974,Imagine you're helping a friend who failed a t...
5,Scenario-based,3,0.836215,Imagine you're helping a friend who failed a t...
6,Persona-based,1,0.857517,"As a wise monk, write a quote about inner stre..."
7,Persona-based,2,0.841372,"As a wise monk, write a quote about inner stre..."
8,Persona-based,3,0.859363,"As a wise monk, write a quote about inner stre..."
9,Keyword-based,1,0.837028,"Using the words 'growth', 'struggle', and 'hop..."


In [8]:
# Save results to CSV
results_table.to_csv("prompt_engineering_results.csv", index=False)
print("\nResults saved to 'prompt_engineering_results.csv'")


Results saved to 'prompt_engineering_results.csv'


In [9]:
# Print submission checklist
print("\n=== Submission Checklist ===")
print("1. The 5 prompts you designed:")
for i, prompt in enumerate(prompts, 1):
    print(f"   {i}. Type: {prompt['type']}")
    print(f"      Text: {prompt['text']}")

print("\n2. All 15 GPT-2 outputs (3 per prompt):")
print("   (See generated_texts in the CSV file)")

print("\n3. 1 human-written reference:")
print(f"   Text: {human_reference['text']}")
print(f"   Source: {human_reference['source']}")

print("\n4. BERTScore output table:")
print("   (See BERTScore F1 column in the CSV file)")


=== Submission Checklist ===
1. The 5 prompts you designed:
   1. Type: Direct instruction
      Text: Write a motivational quote about overcoming fear.
   2. Type: Scenario-based
      Text: Imagine you're helping a friend who failed a test. Write something encouraging.
   3. Type: Persona-based
      Text: As a wise monk, write a quote about inner strength.
   4. Type: Keyword-based
      Text: Using the words 'growth', 'struggle', and 'hope', write something inspiring.
   5. Type: Conversational
      Text: User: I feel like giving up. GPT-2: Here's a quote for you:

2. All 15 GPT-2 outputs (3 per prompt):
   (See generated_texts in the CSV file)

3. 1 human-written reference:
   Text: The only way to overcome fear is to face it. Every struggle you endure plants the seeds for future growth. Have hope, for even the darkest night will end and the sun will rise.
   Source: https://example.com/motivational-quotes

4. BERTScore output table:
   (See BERTScore F1 column in the CSV file)
